**Problem Statement**
The goal here is to predict the Price of a refurbished car based on the variables provided in the data set by building a regression model

# **Data Cleaning**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import mpl_toolkits
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn import preprocessing
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn import metrics
import warnings 
warnings.filterwarnings('ignore')



WE IMPORT THE DATE SET AND ANALYSE THE FEATURES

In [ ]:
car_train=pd.read_csv("Training_Data_Set.csv")
car_train.head()

,Id,Maker,model,Location,Distance,Owner Type,manufacture_year,Age of car,engine_displacement,engine_power,body_type,Vroom Audit Rating,transmission,door_count,seat_count,fuel_type,Price
0,25001,skoda,octavia,Ahmedabad,NaN,Second,1964,55,1964,147.0,compact,8,man,None,None,petrol,543764.25
1,25002,fiat,panda,Ahmedabad,27750.0,Third,2012,7,1242,51.0,NaN,6,man,4,4,petrol,401819.25
2,25003,bmw,x1,Hyderabad,46000.0,Third,2014,5,1995,105.0,NaN,7,auto,4,5,diesel,2392855.50
3,25004,nissan,juke,Mumbai,43949.0,Third,2011,8,1618,140.0,NaN,7,man,4,5,petrol,958606.50
4,25005,bmw,x5,Jaipur,59524.0,Fourth & Above,2012,7,2993,180.0,NaN,7,auto,4,5,diesel,3085561.50


CHECKING THE DATA TYPES 

In [ ]:
car_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53515 entries, 0 to 53514
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   53515 non-null  int64  
 1   Maker                53515 non-null  object 
 2   model                53515 non-null  object 
 3   Location             53515 non-null  object 
 4   Distance             52304 non-null  float64
 5   Owner Type           53515 non-null  object 
 6   manufacture_year     53515 non-null  int64  
 7   Age of car           53515 non-null  int64  
 8   engine_displacement  53515 non-null  int64  
 9   engine_power         52076 non-null  float64
 10  body_type            4136 non-null   object 
 11  Vroom Audit Rating   53515 non-null  int64  
 12  transmission         53515 non-null  object 
 13  door_count           53515 non-null  object 
 14  seat_count           53515 non-null  object 
 15  fuel_type            53515 non-null 

CHECKING FOR NULL VALUES

In [ ]:
car_train.isnull().sum()

Id                         0
Maker                      0
model                      0
Location                   0
Distance                1211
Owner Type                 0
manufacture_year           0
Age of car                 0
engine_displacement        0
engine_power            1439
body_type              49379
Vroom Audit Rating         0
transmission               0
door_count                 0
seat_count                 0
fuel_type                  0
Price                      0
dtype: int64

AS THE FEATURE "body_type" IS HAVING SO MANY NULL VALUES IT IS NOT WORTH REPLACING SO WE DROP IT

In [ ]:
car_train=car_train.drop(['body_type'],axis=1)

AS THE FEATURES, "Id","model", "Location", "manufacture_year" ARE NOT GIVING ANY NEW INFORMATION WE DROP THOSE .

In [ ]:
car_train.drop(['Id','model','Location','manufacture_year'],axis=1,inplace=True)

In [ ]:
car_train.shape

(53515, 12)

In [ ]:
car_train.describe()

,Distance,Age of car,engine_displacement,engine_power,Vroom Audit Rating,Price
count,5.230400e+04,53515.000000,53515.000000,52076.000000,53515.000000,5.351500e+04
mean,9.454626e+04,8.591890,1904.049014,100.448345,5.998374,1.098084e+06
std,2.755617e+05,4.650322,1496.564596,45.330622,1.418336,8.441565e+05
min,0.000000e+00,3.000000,14.000000,10.000000,4.000000,3.000000e+00
25%,1.549000e+04,5.000000,1395.000000,73.000000,5.000000,5.051812e+05
50%,6.552000e+04,8.000000,1896.000000,91.000000,6.000000,8.854552e+05
75%,1.356410e+05,11.000000,1995.000000,125.000000,7.000000,1.477829e+06
max,9.899800e+06,85.000000,32000.000000,896.000000,8.000000,2.212078e+07


In [ ]:
car_train.select_dtypes(include=object).head(6)

,Maker,Owner Type,transmission,door_count,seat_count,fuel_type
0,skoda,Second,man,None,None,petrol
1,fiat,Third,man,4,4,petrol
2,bmw,Third,auto,4,5,diesel
3,nissan,Third,man,4,5,petrol
4,bmw,Fourth & Above,auto,4,5,diesel
5,skoda,First,man,4,5,diesel


REPLACING NULL VALUES

In [ ]:
car_train.door_count.replace('None', np.NaN, inplace = True)
car_train.seat_count.replace('None', np.NaN, inplace = True)
car_train.door_count = car_train.door_count.astype('float')
car_train.seat_count = car_train.seat_count.astype('float')

In [ ]:
car_train['Distance '].fillna(car_train['Distance '].median(), inplace = True)
car_train['engine_power'].fillna(car_train['engine_power'].median(), inplace = True)
car_train.door_count.fillna(int(car_train['door_count'].mode()), inplace = True)
car_train.seat_count.fillna(int(car_train['seat_count'].mode()), inplace = True)

# **ENCODING**

In [ ]:
car_train['Owner Type'] = car_train['Owner Type'].replace({'First':1,'Second':2,'Third':3,'Fourth & Above':4})


In [ ]:
car_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53515 entries, 0 to 53514
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Maker                53515 non-null  object 
 1   Distance             53515 non-null  float64
 2   Owner Type           53515 non-null  int64  
 3   Age of car           53515 non-null  int64  
 4   engine_displacement  53515 non-null  int64  
 5   engine_power         53515 non-null  float64
 6   Vroom Audit Rating   53515 non-null  int64  
 7   transmission         53515 non-null  object 
 8   door_count           53515 non-null  float64
 9   seat_count           53515 non-null  float64
 10  fuel_type            53515 non-null  object 
 11  Price                53515 non-null  float64
dtypes: float64(5), int64(4), object(3)
memory usage: 4.9+ MB


In [ ]:
car_train['Maker'].unique()

array(['skoda', 'fiat', 'bmw', 'nissan', 'audi', 'toyota', 'hyundai',
       'maserati'], dtype=object)

In [ ]:
car_train['transmission'].unique()

array(['man', 'auto'], dtype=object)

In [ ]:
car_train['fuel_type'].unique()

array(['petrol', 'diesel'], dtype=object)

In [ ]:
car=pd.get_dummies(car_train, drop_first = True)
car

,Distance,Owner Type,Age of car,engine_displacement,engine_power,Vroom Audit Rating,door_count,seat_count,Price,Maker_bmw,Maker_fiat,Maker_hyundai,Maker_maserati,Maker_nissan,Maker_skoda,Maker_toyota,transmission_man,fuel_type_petrol
0,65520.0,2,55,1964,147.0,8,4.0,5.0,543764.25,0,0,0,0,0,1,0,1,1
1,27750.0,3,7,1242,51.0,6,4.0,4.0,401819.25,0,1,0,0,0,0,0,1,1
2,46000.0,3,5,1995,105.0,7,4.0,5.0,2392855.50,1,0,0,0,0,0,0,0,0
3,43949.0,3,8,1618,140.0,7,4.0,5.0,958606.50,0,0,0,0,1,0,0,1,1
4,59524.0,4,7,2993,180.0,7,4.0,5.0,3085561.50,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53510,29334.0,4,5,1598,77.0,4,4.0,5.0,1342996.50,0,0,0,0,0,1,0,1,0
53511,223631.0,4,10,1900,77.0,8,5.0,5.0,510732.75,0,0,0,0,0,1,0,1,0
53512,25500.0,3,4,1995,105.0,4,4.0,5.0,2008123.50,1,0,0,0,0,0,0,0,0
53513,1195500.0,3,8,11950,93.0,5,4.0,5.0,874352.25,0,0,0,0,0,0,1,1,0


In [ ]:
car.shape

(53515, 18)

# **SPLITTING DATA INTO TRAIN AND TEST**

In [ ]:
X = car.drop('Price',axis=1)
y = car['Price']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=9)

SCALING

In [ ]:
scaler=StandardScaler()
X_train_scale=scaler.fit_transform(X_train)
X_test_scale=scaler.transform(X_test)

# **linear regression**

In [ ]:
Lregressor=LinearRegression(fit_intercept=True)
Lregressor.fit(X_train_scale,y_train)

LinearRegression()

In [ ]:
y_pred=Lregressor.predict(X_test_scale)

In [ ]:
LRr2 = r2_score(y_test,y_pred)
LRrmse = np.sqrt(mean_squared_error(y_test,y_pred))
print(LRr2)
print(LRrmse)

0.7476280295671998
423958.47825121775


**LINEAR REGRESSION**

**R^2=0.74**






**RMSE=423958.47**

# **Decision Tree**

In [ ]:
Dregressor = DecisionTreeRegressor()
Dregressor.fit(X_train_scale, y_train)

DecisionTreeRegressor()

In [ ]:
dt_pred=Dregressor.predict(X_test_scale)


In [ ]:
DTr2 = r2_score(y_test,dt_pred)
DTrmse = np.sqrt(mean_squared_error(y_test,dt_pred))
print(DTr2)
print(DTrmse)

0.8566585621323275
319513.08144191303


**DECISION TREE**

**R^2=0.85**






**RMSE=319513.08**

# **Random Forest**

In [ ]:
rf=RandomForestRegressor(n_estimators=200,random_state=9)
rf.fit(X_train_scale,y_train)

RandomForestRegressor(n_estimators=200, random_state=9)

In [ ]:
rf_pred=rf.predict(X_test_scale)


In [ ]:
RFr2 = r2_score(y_test,rf_pred)
RFrmse = np.sqrt(mean_squared_error(y_test,rf_pred))
print(RFr2)
print(RFrmse)

0.9470290312237446
194232.42090616838


**RANDOM FOREST**

**R^2=0.94**






**RMSE=194232.42**

# **XGBOOST**

In [ ]:
XG_boost=xgb.XGBRegressor()
XG_boost.fit(X_train_scale,y_train)

XGBRegressor()

In [ ]:
xg_pred=XG_boost.predict(X_test_scale)

In [ ]:
XGr2 = r2_score(y_test,xg_pred)
XGrmse = np.sqrt(mean_squared_error(y_test,xg_pred))
print(XGr2)
print(XGrmse)

0.9270737101665797
227900.26225062055


**XGBOOST**

**R^2=0.92**






**RMSE=227900.26**

# **CONCLUSION**

**RANDOM FOREST IS THE BEST MODEL FOR PREDICTING CAR PRICE WITH THE GIVEN FEATURES BASED ON THE FOLLOWING METRICS**


**R^2=0.94**






**RMSE=194232.42** 